In [129]:
import pandas as pd
pd.options.display.max_columns = 500

In [130]:
nc = pd.read_excel('input/Base NC y SS.xlsx', sheet_name= 'NC', dtype = str)

In [144]:
nc.loc[nc['COD AUT NC'] == '081815']

,BASE,SS,CC,OC,F12,RED,4 DIG TARJETA,FECHA TRANSACCION,COD AUTORIZACION,VALOR ORIGINAL DE COMPRA,TIPO DE DEVOLUCION,VALOR NUEVA COMPRA,VALOR DE DEVOLUCION,FECHA DE INGRESO TESOR.,FECHA ENVIO A RED,ESTADO,FECHA APROBACION DE LA RED,TIPOLOGIA DE RECHAZO,MOTIVO DE RECHAZO,OBSERVACION RECHAZO,AREA DE RECHAZO,DIRIGIDO A,LIQUIDACION,FECHA LIQUIDACION,N° BASE,BASE2,COD AUT NC,ID Registro,FECHA,POS,SECUENCIA,TIENDA,MONTO,ESTADO F12,UBICACIÓN,MEDIO DE PAGO,MONTO TOTAL,MOTIVO,EMAIL,SKUS o F12,EAN,UND,Area,Aplica Cambio estado CMP,Dias Estimada de Rta,Dias de Liquidacion,LOCAL
78975,CMR UNO A UNO REPROCESOS CONCILIACION 27072022...,1-126465263797,79739596,LINK DE PAGO,12691321715,CMR,3148,2022-05-27 00:00:00,081815,835900,TOTAL,0,835900,2022-07-27 00:00:00,2022-07-28 00:00:00,PROCESADA,2022-08-02 00:00:00,0,0,0,0,CMR,PROCESADA LIQUID,2022-08-03 00:00:00,37639,CMR UNO A UNO REPROCESOS CONCILIACION 27072022...,081815,485145738,2022-06-10 00:00:00,62,2929,3000,835900,ANULADO X NCDR,0,CMR,835900,INCUMPLIMIENTO,anguloruben@hotmail.com,12691321715,0,0,Uno a uno,0,-1,0,NaN


In [145]:
nc['COD AUT NC'] = nc['COD AUT NC'].str.strip()

In [146]:
nc['VALOR DE DEVOLUCION'] = nc['VALOR DE DEVOLUCION'].str.replace('36286.5', '36286')

AttributeError: Can only use .str accessor with string values!

In [147]:
nc['VALOR DE DEVOLUCION'] = nc['VALOR DE DEVOLUCION'].astype(int)

In [148]:
nc_proc_cm = nc.loc[(nc['ESTADO'] == 'PROCESADA') & (nc['COD AUT NC'] == '10009')]

In [149]:
nc_proc_cm.drop_duplicates('SS', inplace=True)

C:\Users\maperezr\AppData\Local\Temp\ipykernel_30852\516989417.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nc_proc_cm.drop_duplicates('SS', inplace=True)


In [150]:
nc_proc_cm = nc_proc_cm[['SS', 'COD AUTORIZACION', 'MONTO TOTAL', '4 DIG TARJETA']]

In [151]:
nc_proc = nc.loc[(nc['ESTADO'] == 'PROCESADA') & (nc['COD AUT NC'] != '10009')]

In [152]:
nc_proc.drop_duplicates(['COD AUT NC', 'COD AUTORIZACION', 'MONTO TOTAL', '4 DIG TARJETA'], inplace = True)

C:\Users\maperezr\AppData\Local\Temp\ipykernel_30852\2249370976.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nc_proc.drop_duplicates(['COD AUT NC', 'COD AUTORIZACION', 'MONTO TOTAL', '4 DIG TARJETA'], inplace = True)


In [153]:
nc_gb_proc = nc_proc.groupby(['COD AUT NC','ESTADO'])['VALOR DE DEVOLUCION'].sum().reset_index()

In [154]:
caut_nc = nc_proc.groupby(['COD AUT NC','ESTADO'])['VALOR DE DEVOLUCION'].sum().reset_index()['COD AUT NC']

NO PROCESADAS

In [155]:
nc_no_proc = nc.loc[(nc['ESTADO'] != 'PROCESADA') & (nc['COD AUT NC'] != '10009') ]

In [156]:
nc_no_proc.drop_duplicates('COD AUT NC', inplace = True)

C:\Users\maperezr\AppData\Local\Temp\ipykernel_30852\3352299271.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nc_no_proc.drop_duplicates('COD AUT NC', inplace = True)


In [157]:
nc_no_proc = nc_no_proc[['COD AUT NC','ESTADO','VALOR DE DEVOLUCION']]

In [158]:
nc_no_proc = nc_no_proc.loc[~nc_no_proc['COD AUT NC'].isin(caut_nc)]

In [159]:
nc_no_proc['COD AUT NC'].nunique()

3393

In [160]:
nc['COD AUT NC'].nunique()

78389

In [161]:
final_nc = pd.concat([nc_gb_proc, nc_no_proc])

In [162]:
final_nc['COD AUT NC'].nunique()

78388

In [163]:
nc.loc[nc['COD AUT NC'] != "10009"]['COD AUT NC'].nunique()

78388

In [164]:
final_nc.to_csv('221127_tesoreria_ntc.csv', index = False, sep = ';')

In [165]:
final_nc

,COD AUT NC,ESTADO,VALOR DE DEVOLUCION
0,081815,PROCESADA,835900
1,1003169734,PROCESADA,187480
2,1013577389,PROCESADA,3041019
3,1013621545,PROCESADA,269930
4,1047260785,PROCESADA,169990
...,...,...,...
79188,1075723894 -1075723894,RECHAZADA,90731
79218,1082210509,RECHAZADA,400780
79219,1082211544,RECHAZADA,78000
79220,1082218444,RECHAZADA,496792


SS

In [166]:
ss = pd.read_excel('input/Base NC y SS.xlsx', sheet_name= 'SS', dtype = str)

In [167]:
ss_proc = ss.loc[(ss['ESTADO'] == 'PROCESADA') & (ss['SS'] != '0')]

In [168]:
ss_proc['SS'].nunique()

10862

In [169]:
ss_proc['VALOR DE DEVOLUCION'] = ss_proc['VALOR DE DEVOLUCION'].astype(int)

C:\Users\maperezr\AppData\Local\Temp\ipykernel_30852\465985516.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ss_proc['VALOR DE DEVOLUCION'] = ss_proc['VALOR DE DEVOLUCION'].astype(int)


In [170]:
ss_proc.drop_duplicates(['SS', 'COD AUTORIZACION', 'MONTO TOTAL', '4 DIG TARJETA'], inplace = True)

C:\Users\maperezr\AppData\Local\Temp\ipykernel_30852\989855474.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ss_proc.drop_duplicates(['SS', 'COD AUTORIZACION', 'MONTO TOTAL', '4 DIG TARJETA'], inplace = True)


In [171]:
ss_gb_proc = ss_proc.groupby(['SS','ESTADO'])['VALOR DE DEVOLUCION'].sum().reset_index()
caut_ss = ss_proc.groupby(['SS','ESTADO'])['VALOR DE DEVOLUCION'].sum().reset_index()['SS']

In [172]:
10862 + 984

11846

In [173]:
ss_gb_proc.shape

(10862, 3)

In [174]:
ss_no_proc = ss.loc[(ss['ESTADO'] != 'PROCESADA') & (ss['SS'] != 0)]

In [175]:
ss_no_proc['SS'].nunique()

985

In [176]:
ss_no_proc.drop_duplicates('SS', inplace = True)

C:\Users\maperezr\AppData\Local\Temp\ipykernel_30852\4051014238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ss_no_proc.drop_duplicates('SS', inplace = True)


In [177]:
ss_no_proc = ss_no_proc[['SS','ESTADO','VALOR DE DEVOLUCION']]

In [178]:
ss_no_proc = ss_no_proc.loc[~ss_no_proc['SS'].isin(caut_ss)]

In [179]:
final_ss = pd.concat([ss_gb_proc, ss_no_proc, nc_proc_cm])

In [180]:
final_ss.to_csv('221127_tesoreria_sieb.csv', index = False, sep = ';')